# ML, Data Analysis
### Machine learning: Decision tree, complete code
The **decision tree** is a supervised model used in machine learning for both classification and regression tasks. It models decisions and their possible consequences in a tree-like structure (often binary tree). It has three kinds of nodes:
 - **Root node:** Represents the whole dataset, and like internal nodes it has a decision point.
 - **Internal nodes:** Represents a decision point based on a specific feature.
 - **Leaf nodes:** Represents the final output, a class label for *classification*, or a continuous value for *regression*.
<hr>

**Hint 1:** For *classification*, the value of a **leaf node** is the class label assigned based on the majority class of the training samples in that node. For *regression*, the value of the leaf node is often the **average value** of the samples in that node.
<br>**Hint 2:** Each node of a decision tree except leaf nodes has some **branches**. These branches are the outcomes of the decisions, leading to further internal or leaf nodes. 
<br> **Hint 3:** A decision tree selects the best feature to split the data at each node based on a diversity measure (e.g., Gini impurity, informstion gain for classification, or mean squared error (mse) for regression). We call this process **node splitting** which was covered fully in an earlier post.
<hr>

The **decision tree algorithm** may be specified as:
    <br>1) Start at the root node with the full dataset. 
    <br>2) For each feature, evaluate all possible splits:
        <br>&nbsp; &nbsp; - *Classification*: Minimize weighted Gini impurity or entropy.
        $Gini=1−\sum_i {p_i}^2$ (or $Entropy=−\sum_i p_i log p_i$).
        <br>&nbsp; &nbsp; &nbsp; For example, the Gini impurity after the split becomes:
        <br> &nbsp; &nbsp; $Gini_{split}=\frac{∣X_{left}∣}{∣X∣}\cdot Gini(X_{left})+\frac{∣X_{right}∣}{∣X∣}\cdot Gini(X_{right})$
        <br>&nbsp; &nbsp; - *Regression*: Minimize weighted MSE (equivalent to variance).
        <br> &nbsp; &nbsp; $MSE_{split}=\frac{∣X_{left}∣}{∣X∣}\cdot Var(X_{left})+\frac{∣X_{right}∣}{∣X∣}\cdot Var(X_{right})$
    <br>3) Select the best split (feature + threshold) that maximizes purity gain.
    <br>4) Partition data into left/right child nodes.
    <br>5) Repeat recursively until stopping:
        - All samples in a node belong to the same class (classification) or have the same value (regression).
        - Maximum depth reached.
        - Minimum samples per node.
       <br>6) Leaf nodes predict:
        <br> &nbsp; - Classification: Majority class.
        <br> &nbsp; - Regression: Mean/median of samples.
<hr>

**Hint 4:** In the formulae, the symbol $|\cdot|$ returns the number of elements of the given set.
<hr>
In the following, we use train a decision tree with Iris dataset for classifiction task. We use the Gini impurity for splitting the dataset at nodes

<hr>
https://github.com/ostad-ai/Machine-Learning
<br> Explanation: https://www.pinterest.com/HamedShahHosseini/machine-learning/algorithms-with-python-codes/

In [1]:
# Load required modules
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split

In [2]:
# Define the tree node class
class TreeNode:
    def __init__(self, feature_idx=None, threshold=None,
                      left=None, right=None, value=None):
        self.feature_idx = feature_idx  # Index of feature to split on
        self.threshold = threshold      # Threshold value for the split
        self.left = left                # Left subtree (≤ threshold)
        self.right = right              # Right subtree (> threshold)
        self.value = value              # Class label (for leaf nodes)

In [3]:
# Define the decision tree class
# with maximum depth, and minimum samples to split a node
# The task can be chosen to be classification or regression
class DecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2, task="classification"):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.task = task  # "classification" or "regression"
        self.root = None
    
    # Train (build) the tree with training data pairs
    def fit(self, X, y):
        self.root = self._build_tree(X, y, depth=0)
    
    # build the tree with maximum depth and min. samples
    def _build_tree(self, X, y, depth):
        # Stopping conditions
        if (depth >= self.max_depth or 
            len(y) < self.min_samples_split or 
            len(set(y)) == 1):
            return self._create_leaf_node(y)
        
        # Find best split
        feature_idx, threshold = self._find_best_split(X, y)
        if feature_idx is None:
            return self._create_leaf_node(y)
        
        # Split data
        left_mask = X[:, feature_idx] <= threshold
        right_mask = ~left_mask
        
        # Recursively build subtrees
        left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)
        return TreeNode(feature_idx, threshold, left_subtree, right_subtree)
    
    # Create a leaf node with a value of instance TreeNode
    def _create_leaf_node(self, y):
        if self.task == "classification":
            counts = Counter(y)
            # Most frequent class
            value = counts.most_common(1)[0][0]
        else:  # regression
            value = np.mean(y)
        return TreeNode(value=value)

    # Find the best (binary) split for the given data pairs
    def _find_best_split(self, X, y):
        best_gini = float('inf')
        best_feature, best_threshold = None, None
        
        for feature_idx in range(X.shape[1]):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                left_mask = X[:, feature_idx] <= threshold
                right_mask = ~left_mask
                
                if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                    continue
                
                if self.task == "classification":
                    gini = self._weighted_gini(y[left_mask], y[right_mask])
                else:  # regression
                    gini = self._weighted_mse(y[left_mask], y[right_mask])
                
                if gini < best_gini:
                    best_gini = gini
                    best_feature = feature_idx
                    best_threshold = threshold
                    
        return best_feature, best_threshold

    # Weighted Gini impurity
    def _weighted_gini(self, y_left, y_right): # for classification
        n_left, n_right = len(y_left), len(y_right)
        n_total = n_left + n_right
        
        gini_left = 1 - sum((np.sum(y_left == c) / n_left) ** 2 for c in set(y_left))
        gini_right = 1 - sum((np.sum(y_right == c) / n_right) ** 2 for c in set(y_right))
        
        return (n_left / n_total) * gini_left + (n_right / n_total) * gini_right
    
    # Weighted variance
    def _weighted_mse(self, y_left, y_right): # for regression
        n_left, n_right = len(y_left), len(y_right)
        n_total = n_left + n_right
        
        mse_left = np.mean((y_left - np.mean(y_left)) ** 2)
        mse_right = np.mean((y_right - np.mean(y_right)) ** 2)
        
        return (n_left / n_total) * mse_left + (n_right / n_total) * mse_right

    # Predict labels for given data matrix X
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    # Predict label for single data point x
    def _traverse_tree(self, x, node):
        if node.value is not None: # node is a leaf
            return node.value
        if x[node.feature_idx] <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)

    # Print the structure of the tree
    def print_tree(self, node=None, indent=0):
        if node is None:
            node = self.root
        prefix = "    " * indent
        if node.value is not None:
            print(f"{prefix}\u25cb Leaf: {node.value}")
        else:
            print(f"{prefix}\u25c9 Feature_{node.feature_idx} ≤ {node.threshold}")
            if hasattr(node, "left"):
                print(f"{prefix}\u27a9 True:")
                self.print_tree(node.left, indent + 1)
            if hasattr(node, "right"):
                print(f"{prefix}\u27a9 False:")
                self.print_tree(node.right, indent + 1)

In [4]:
# Load the Iris dataset into a Pandas dataframe
df=pd.read_csv('https://raw.githubusercontent.com/ostad-ai/Machine-Learning/refs/heads/main/iris.csv')

In [5]:
# Show the top rows of the dataframe
df.head()

,Sepal Length,Sepal Width,Petal Length,Petal Width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [6]:
# Extract data matrix (X) and labels (y)
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train & predict
tree = DecisionTree(max_depth=3,task="classification")
tree.fit(X_train, y_train)
predictions = tree.predict(X_test)

# Evaluate accuracy
accuracy = np.sum(predictions == y_test) / len(y_test)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.97


In [7]:
# Print the structure of the learnt decision tree
tree.print_tree()

◉ Feature_2 ≤ 1.9
➩ True:
    ○ Leaf: Iris-setosa
➩ False:
    ◉ Feature_2 ≤ 4.7
    ➩ True:
        ◉ Feature_3 ≤ 1.6
        ➩ True:
            ○ Leaf: Iris-versicolor
        ➩ False:
            ○ Leaf: Iris-virginica
    ➩ False:
        ◉ Feature_3 ≤ 1.7
        ➩ True:
            ○ Leaf: Iris-virginica
        ➩ False:
            ○ Leaf: Iris-virginica
